Importing all necessary Python libraries

In [67]:
import requests
import os
import os.path
import zipfile
import sqlite3
import csv
import io
import pandas as pd
import pandas as pd
from pandas import ExcelWriter

Creating a staging sub directory called 'staging'

In [68]:
staging_dir_name = "staging"
os.mkdir(staging_dir_name)
os.path.isdir(staging_dir_name)

True

Download zip file from the internet

In [69]:
url = "https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"
r = requests.get(url)
zip_file_name = os.path.join(staging_dir_name, "Hospital_Revised_Flatfiles.zip")
zf = open(zip_file_name, "wb")
zf.write(r.content)
zf.close()

Unzipping the zip files

In [70]:
z = zipfile.ZipFile(zip_file_name,"r")
z.extractall(staging_dir_name)
z.close()


Creating a function to standardize all the table and column names

In [71]:
def name_standardize(name,torc = "t"):
    name.strip()
    name=name.lower()
    name=name.replace(" ","_").replace(",","_").replace("\t","_").replace("/","_").replace("-","_").replace("%","pct").replace("#","_num")
    if torc == 't':
        if not name[0].isalpha():
            name="t_"+name
    if torc == 'c':
        if not name[0].isalpha():
            name="c_"+name
    return name

In [72]:
wd = os.getcwd()
staging_path = wd + '//staging'

Encoding all the files to utf8

In [73]:
os.chdir(staging_path)
files = os.listdir()
for file in files:
    if not file.endswith(".csv"):
        os.remove(file)
        continue
    base_file = file + "-base"
    os.rename(file,base_file)
    new_name=name_standardize(file,"t")
    with io.open(base_file, 'r', encoding='cp1252') as f_in:
        f_in = csv.reader(x.replace('\0', '') for x in f_in)
        with io.open(new_name, 'w', encoding="utf8", newline='') as f_out:
            f_out = csv.writer(f_out,delimiter=',')
            for i, line in enumerate(f_in):
                if line == [] or line == [' ']:
                    continue
                elif i == 0:
                    s = []
                    for x in range(0,len(line)):
                        s.append(name_standardize(line[x],"c"))
                    f_out.writerow([str(elem) for elem in s])
                else:
                    f_out.writerow([str(elem) for elem in line])
os.chdir(wd)

Creating a database 'medicare_hospital_compare'

In [74]:
db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()

Creating tables in the database and inserting data into it

In [75]:
os.chdir(staging_path)
tablenames = os.listdir()
for table in tablenames:
    if table.endswith(".csv-base"):
        continue
    else:
        tablename = table.rstrip('.csv')
        with io.open(table, 'r', encoding="utf8") as t_in:
            table_in = csv.reader(t_in)
            r = next(table_in)
            header = []
            for i in range(0,len(r)):
                header.append(r[i] + ' ' + 'TEXT')
                cols = ','.join(header)
            sql ='CREATE TABLE IF NOT EXISTS ' + tablename + ' (' + cols + ')'
            c.execute(sql)
            insert_stmt = "INSERT INTO " + tablename + '({0})' + ' VALUES({1});'
            query = insert_stmt.format(','.join(r), ','.join('?' * len(r)))
            for line in table_in:
                c.execute(query, line)
os.chdir(wd)
db.commit()
db.close()

Downloading hospital_ranking_focus_states excel from the internet

In [76]:
url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
r = requests.get(url)
with open('ranking.xlsx', 'wb') as output:
    output.write(r.content)

Exporting hospital_ranking_focus_states file to database

In [77]:
xl = pd.ExcelFile("ranking.xlsx")

db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()
file = xl.sheet_names

for sheet in file:
    data = pd.DataFrame()
    header = []
    n_t = name_standardize(sheet)
    data = pd.read_excel(xl,sheetname=sheet, dtype=str) 
    h = list(data.columns.values)
    for value in h:
        header.append(name_standardize(value,"c"))
    data.columns = header
    data.applymap(str)
    data.to_sql(n_t, db, if_exists="replace",index=False)

db.commit()
db.close()

Generating tables in database for getting required information

In [78]:
db = sqlite3.connect('medicare_hospital_compare.db')
c = db.cursor()

c.execute('drop table if exists newtable')
c.execute('create table newtable as select b.ranking, a.provider_id, a.hospital_name, a.city, a.state, a.county_name from hospital_general_information a join hospital_national_ranking b on a.provider_id = b.provider_id')
c.execute('drop table if exists final_table')
c.execute('create table final_table AS select b.*, a.state_name FROM focus_states a join newtable b on b.state = a.state_abbreviation')
db.commit

<function Connection.commit>

Creating databases from all these newly created table

In [79]:
national_df = pd.read_sql_query("select * from newtable;", db)
final_df = pd.read_sql_query("select * from final_table;", db)
measure_df = pd.read_sql_query("select state, measure_id, measure_name, score from timely_and_effective_care___hospital", db)
db.close()

Writing the information of all the required columns into a new excel file named hospital_ranking

In [80]:
states= list(final_df.state_name.unique())
abriv= list(final_df.state.unique())
state_dict = dict(zip(abriv, states))
final_df[['ranking']] = final_df[['ranking']].astype(int)
national_df[['ranking']] = national_df[['ranking']].astype(int)
title = ['Provider ID','Hospital Name','City','State','County']

writer = pd.ExcelWriter('hospital_ranking.xlsx')

Nationwide = national_df.sort_values('ranking')
Nationwide = Nationwide.head(n=100)
del Nationwide['ranking']
Nationwide.columns = title
Nationwide.to_excel(writer,'Nationwide',index=False)

In [81]:
d = {}
for state in states:
    temp = final_df[final_df['state_name'] == state]
    d[state] = temp.sort_values('ranking')
    d[state] = d[state].head(n=100)
    del d[state]['state_name']
    del d[state]['ranking']
    d[state].columns = title
    d[state].to_excel(writer,state,index=False)
writer.save()
writer.close()

Getting the summary statistics like min, max, average and standard deviation from the data and writing it to measure_statistics file

In [82]:
measures = measure_df[measure_df['score'].astype(str).str.isdigit()]
measures['score'] = measures['score'].apply(pd.to_numeric, errors='coerce')

writer2 = pd.ExcelWriter('measure_statistics.xlsx')
nation_stat =  measures.groupby(['measure_id','measure_name'])
summary_statistics = nation_stat['score'].describe()
del summary_statistics['count']
del summary_statistics['25%']
del summary_statistics['50%']
del summary_statistics['75%']
summary_statistics = summary_statistics[['min','max','mean','std']]
summary_statistics['std'].fillna(0, inplace=True)
summary_statistics=summary_statistics.reset_index()
col_name = ['Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation']
summary_statistics.columns = col_name
summary_statistics.to_excel(writer2,'Nationwide',index=False)

C:\Users\sanke\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Grouping the data by states and finding statistics for each individual state and writing it into different excel sheets

In [83]:

grouped_data = measures.groupby(['state', 'measure_id','measure_name'])
state_summary = grouped_data['score'].describe()
del state_summary['count']
del state_summary['25%']
del state_summary['50%']
del state_summary['75%']
state_summary = state_summary[['min','max','mean','std']]
state_summary['std'].fillna(0, inplace=True)
state_summary=state_summary.reset_index()
col_name = ['state', 'Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation']
state_summary.columns = col_name

for a,f in state_dict.items():
    temp = state_summary[state_summary['state'] == a]
    del temp['state']
    temp.to_excel(writer2,f,index=False)
writer2.save()
writer2.close()